## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-12-08-24-19 +0000,cbc,Ontario Sikh leader explains why he refused pr...,https://www.cbc.ca/news/politics/ontario-sikh-...
1,2025-10-12-08-12-57 +0000,nypost,Moment iconic replica of Christopher Columbus ...,https://nypost.com/2025/10/12/world-news/marig...
2,2025-10-12-07-32-58 +0000,bbc,How will the EU's new border system work?,https://www.bbc.com/news/articles/c39rkpe8mj2o...
3,2025-10-12-07-26-03 +0000,nyt,China Flexed. Trump Hit Back. So Much for the ...,https://www.nytimes.com/2025/10/11/world/asia/...
4,2025-10-12-07-22-45 +0000,nypost,"Chinese coast guard fires water cannon, rams P...",https://nypost.com/2025/10/12/world-news/chine...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2375/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
29,trump,29
134,gaza,16
203,shutdown,11
58,tennessee,9
27,china,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
118,2025-10-11-17-17-44 +0000,nypost,Putin chides Nobel winners who ‘did nothing fo...,https://nypost.com/2025/10/11/world-news/putin...,81
112,2025-10-11-18-12-00 +0000,nypost,President Trump will make sure military troops...,https://nypost.com/2025/10/11/us-news/presiden...,80
91,2025-10-11-20-36-00 +0000,wsj,Trump said funds have been identified to pay m...,https://www.wsj.com/politics/policy/trump-says...,79
15,2025-10-12-04-37-28 +0000,bbc,Israelis praise Trump at huge rally ahead of e...,https://www.bbc.com/news/articles/c75q5qng0rgo...,74
155,2025-10-11-12-30-00 +0000,nypost,First hostage freed from Hamas’ grasp thanks T...,https://nypost.com/2025/10/11/world-news/first...,72


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
118,81,2025-10-11-17-17-44 +0000,nypost,Putin chides Nobel winners who ‘did nothing fo...,https://nypost.com/2025/10/11/world-news/putin...
112,51,2025-10-11-18-12-00 +0000,nypost,President Trump will make sure military troops...,https://nypost.com/2025/10/11/us-news/presiden...
111,41,2025-10-11-18-19-42 +0000,cbc,"Tennessee authorities confirm 16 dead, no surv...",https://www.cbc.ca/news/world/tennessee-explos...
4,37,2025-10-12-07-22-45 +0000,nypost,"Chinese coast guard fires water cannon, rams P...",https://nypost.com/2025/10/12/world-news/chine...
155,35,2025-10-11-12-30-00 +0000,nypost,First hostage freed from Hamas’ grasp thanks T...,https://nypost.com/2025/10/11/world-news/first...
145,30,2025-10-11-13-44-51 +0000,nypost,Joe Biden undergoing radiation in new phase of...,https://nypost.com/2025/10/11/us-news/joe-bide...
65,28,2025-10-11-22-45-15 +0000,nypost,"Argentinian model, singer Fede Dorcaz shot and...",https://nypost.com/2025/10/11/world-news/argen...
79,27,2025-10-11-21-29-39 +0000,startribune,St. Paul police: 22-year-old man was assaulted...,https://www.startribune.com/st-paul-police-man...
29,25,2025-10-12-02-45-06 +0000,latimes,Former schools chief Austin Beutner plans to c...,https://www.latimes.com/california/story/2025-...
91,25,2025-10-11-20-36-00 +0000,wsj,Trump said funds have been identified to pay m...,https://www.wsj.com/politics/policy/trump-says...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
